##Importing Libraries and Datasets

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Final_DSNotes/Projects/ML_Projects/Amazon_product_Recommender/Myntra_fashion_products.csv')
df.head(2)
data = df.copy()

In [ ]:
df.drop(columns=['sku','mpn','price','in_stock','currency'],inplace=True)

##Text Preprocessing

In [ ]:
brand_counts = df['brand'].value_counts()
brands_to_keep = brand_counts[brand_counts > 20].index

df = df[df['brand'].isin(brands_to_keep)]
df['brand'].value_counts()

,count
brand,
Indian Terrain,971
Puma,345
Pepe Jeans,340
AURELIA,307
Flying Machine,301
...,...
Kazo,21
MISH,21
INCOLOR,21


In [ ]:
df.reset_index(inplace=True)
df

,index,name,brand,description,images,gender
0,0,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,"Black and grey printed medium trolley bag, sec...",http://assets.myntassets.com/assets/images/100...,Unisex
1,1,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Beige & Grey made to measure kurta with churid...,http://assets.myntassets.com/assets/images/100...,Women
2,2,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Pink coloured wash 5-pocket high-rise cropped ...,http://assets.myntassets.com/assets/images/100...,Women
3,3,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Blue self-design bandhgala suitBlue self-desig...,http://assets.myntassets.com/assets/images/pro...,Men
4,4,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,"Brown and off-white printed casual shirt, has ...",http://assets.myntassets.com/assets/images/pro...,Men
...,...,...,...,...,...,...
9297,12483,Sonari Pack of 2 Full-Coverage Bras celina,Sonari,Pack of two full-coverage bras in whiteEach ha...,http://assets.myntassets.com/v1/image/style/pr...,Women
9298,12484,Pepe Jeans Men Blue Printed Slim Pure Cotton R...,Pepe Jeans,"Blue and Green printed T-shirt, has a round ne...",http://assets.myntassets.com/assets/images/102...,Men
9299,12486,Pepe Jeans Men Black Hammock Slim Fit Low-Rise...,Pepe Jeans,"Black dark wash 5-pocket low-rise jeans, clean...",http://assets.myntassets.com/assets/images/102...,Men
9300,12487,Mochi Women Gold-Toned Solid Heels,Mochi,"A pair of gold-toned open toe heels, has regul...",http://assets.myntassets.com/assets/images/pro...,Women


In [ ]:
df.drop(columns=['index'],inplace=True)
df.head(2)

,name,brand,description,images,gender
0,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,"Black and grey printed medium trolley bag, sec...",http://assets.myntassets.com/assets/images/100...,Unisex
1,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Beige & Grey made to measure kurta with churid...,http://assets.myntassets.com/assets/images/100...,Women


In [ ]:
len(df)

9302

In [ ]:
df.dtypes

,0
name,object
brand,object
description,object
images,object
gender,object


In [ ]:
df.isna().sum()

,0
name,0
brand,0
description,0
images,0
gender,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.columns

Index(['name', 'brand', 'description', 'images', 'gender'], dtype='object')

In [ ]:
df['content'] = df['name'] + ' ' + df['brand'] + ' ' + df['description']

In [ ]:
df.head()

,name,brand,description,images,gender,content
0,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,"Black and grey printed medium trolley bag, sec...",http://assets.myntassets.com/assets/images/100...,Unisex,DKNY Unisex Black & Grey Printed Medium Trolle...
1,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Beige & Grey made to measure kurta with churid...,http://assets.myntassets.com/assets/images/100...,Women,EthnoVogue Women Beige & Grey Made to Measure ...
2,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Pink coloured wash 5-pocket high-rise cropped ...,http://assets.myntassets.com/assets/images/100...,Women,SPYKAR Women Pink Alexa Super Skinny Fit High-...
3,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Blue self-design bandhgala suitBlue self-desig...,http://assets.myntassets.com/assets/images/pro...,Men,Raymond Men Blue Self-Design Single-Breasted B...
4,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,"Brown and off-white printed casual shirt, has ...",http://assets.myntassets.com/assets/images/pro...,Men,Parx Men Brown & Off-White Slim Fit Printed Ca...


In [ ]:
df['content'][0]

'DKNY Unisex Black & Grey Printed Medium Trolley Bag DKNY Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer'

In [ ]:
df['brand'].nunique()

126

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
cleaned_content = []
for i in range(len(df)):
  text = re.sub(r'[^a-zA-Z]',' ',df['content'][i])
  text = text.lower()
  cleaned_content.append(text)
df['content'] = cleaned_content

In [ ]:
df['content'][0]

'dkny unisex black   grey printed medium trolley bag dkny black and grey printed medium trolley bag  secured with a tsa lockone handle on the top and one on the side  has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsone main zip compartment  zip lining  two compression straps with click clasps  one zip compartment on the flap with three zip pocketswarranty    yearswarranty provided by brand owner   manufacturer'

In [ ]:
corpus = []
for i in range(len(df)):

  text = df['content'][i]
  stemmer = PorterStemmer()
  stopword = stopwords.words('english')

  #Tokenizing
  text = word_tokenize(text)

  #Stemming
  text = [stemmer.stem(word) for word in text if not word in set(stopword)]
  text = ' '.join(text)
  corpus.append(text)

In [ ]:
len(corpus)

9302

In [ ]:
df['content'] = corpus
df['content'][0]

'dkni unisex black grey print medium trolley bag dkni black grey print medium trolley bag secur tsa lockon handl top one side trolley retract handl top four corner mount inlin skate wheelson main zip compart zip line two compress strap click clasp one zip compart flap three zip pocketswarranti yearswarranti provid brand owner manufactur'

##Model Building

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(tokenizer=word_tokenize)

def cosine_sim(txt1,txt2):
  tfid_matrix = tfidvectorizer.fit_transform([txt1,txt2])
  return cosine_similarity(tfid_matrix)[0][1]

In [ ]:
def search_product(query):
    stemmer = PorterStemmer()
    stemmed_query = ' '.join([stemmer.stem(word) for word in word_tokenize(query)])

    # Calculate cosine similarity
    df['similarity'] = df['content'].apply(lambda x: cosine_sim(stemmed_query, x))

    # Get the top 10 results
    res = df.sort_values(by='similarity', ascending=False).head(10)[['brand', 'name']]

    return res

In [ ]:
search_product('black shoes for men but red should work well too')

,brand,name
8356,Puma,Puma Men Red & Black Styron Idp Running Shoes
4916,Duke,Duke Men Red & Black Sports Sandals
1053,HERE&NOW,HERE&NOW Men Red & Black Colourblocked Polo Co...
2125,Red Tape,Red Tape Men Black Leather Semiformal Slip-Ons
1844,Red Tape,Red Tape Men Black Leather Formal Slip-Ons
1552,Red Tape,Red Tape Men Black Leather Formal Slip-Ons
1825,Red Tape,Red Tape Men Black Leather Formal Slip-Ons
2262,Campus,Campus Men Black Mesh Running Shoes
2331,Red Tape,Red Tape Men Black Leather Brogues
871,Geox,Geox Men Black Leather Driving Shoes


##Importing

In [ ]:
import pickle

with open('tfidvectorizer.pkl', 'wb') as file:
    pickle.dump(tfidvectorizer, file)


df.to_pickle('myntra_df.pkl')


In [ ]:
# from google.colab import files

# # Download the pickled files
# files.download('tfidvectorizer.pkl')

# files.download('myntra_df.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>